In [14]:
import cv2
from cv2 import aruco
import numpy as np
import msgpack as mp
import msgpack_numpy as mpn
import os
import matplotlib.pyplot as plt

In [15]:
_pth = os.path.dirname(os.getcwd())
_parent_folder = "data"
_calib_folder_name = "calib_aruco_undistort"

_webcam_calib_folder = os.path.join(
    _pth, _parent_folder, "calibration", _calib_folder_name
)
_webcam_calib_video = os.path.join(_webcam_calib_folder, "webcam_color.msgpack")
_webcam_calib_folder = os.path.join(_webcam_calib_folder)
_webcam_calib_pth = os.path.join(_webcam_calib_folder, "webcam_calibration.msgpack")

In [16]:
ARUCO_PARAMETERS = aruco.DetectorParameters()
ARUCO_DICT = aruco.getPredefinedDictionary(aruco.DICT_ARUCO_ORIGINAL)
detector = aruco.ArucoDetector(ARUCO_DICT, ARUCO_PARAMETERS)
markerLength = 0.05
markerSeperation = 0.01

board = aruco.GridBoard(
    size=[1, 1],
    markerLength=markerLength,
    markerSeparation=markerSeperation,
    dictionary=ARUCO_DICT,
)
_video_pth = _webcam_calib_video
_video_file = open(_video_pth, "rb")
_video_data = mp.Unpacker(_video_file, object_hook=mpn.decode)
_video_length = 0

for _frame in _video_data:
    _video_length += 1

_video_file.close()

In [17]:
_video_pth = _webcam_calib_video
_video_file = open(_video_pth, "rb")
_video_data = list(mp.Unpacker(_video_file, object_hook=mpn.decode))[1:]

marker_corners = []
marker_ids = []
counter = 0
rnd = np.random.choice(_video_length, 150, replace=False)
for idx, data_points in enumerate(_video_data):
    corners, ids = data_points

    marker_corners.append(corners)
    marker_ids.append(ids)
    counter += 1

_video_file.close()

In [18]:
rnd = np.random.choice(_video_length, 50, replace=False)

In [19]:
processed_image_points = []
processed_object_points = []
for _f in range(len(marker_corners)):
    if _f in rnd:
        current_object_points, current_image_points = board.matchImagePoints(
            marker_corners[_f], marker_ids[_f]
        )
        try:
            if current_object_points.any() and current_image_points.any():
                if current_object_points.shape == np.zeros((4, 1, 3)).shape:
                    processed_image_points.append(current_image_points)
                    processed_object_points.append(current_object_points)
        except:
            pass

In [20]:
processed_image_points = np.asarray(processed_image_points, dtype=np.float32)
processed_object_points = np.asarray(processed_object_points, dtype=np.float32)

In [21]:
mtx2 = np.eye(3, dtype=np.float32)
dist2 = np.zeros(14, dtype=np.float32)

In [22]:
# a, mtx1, dist1, b, c = cv2.calibrateCamera(
#     processed_object_points,
#     processed_image_points,
#     (1200, 480),
#     mtx2,
#     dist2,
#     flags=calibration_flags,
#     criteria=term_criteria,
# )
ret, mtx1, dist1, _, _ = cv2.calibrateCamera(
    processed_object_points, processed_image_points, (1200, 800), mtx2, dist2
)

In [23]:
import toml

data = toml.load("/home/sujith/Documents/programs/settings.toml")
data["calibration"]["camera_matrix"] = mtx1.tolist()
data["calibration"]["dist_coeffs"] = dist1.tolist()
data["camera"]["resolution"] = (1200, 800)
with open("/home/sujith/Documents/programs/calib_undistort_aruco.toml", "w") as f:
    toml.dump(data, f)

In [24]:
mtx1

array([[875.04155703,   0.        , 680.12234647],
       [  0.        , 866.57886696, 347.30734302],
       [  0.        ,   0.        ,   1.        ]])

In [25]:
ret

0.24498506180171767